In [1]:
#inicjalizacja bibliotek
import pandas as pd
from sympy import symbols, Eq, solve


#GUI
import tkinter as tk
from tkinter import messagebox

from tkinter.ttk import Combobox
from tkinter import IntVar

In [2]:
class sample:
    def __init__(self, Element1, Element2, concentration_Element1, total_thickness, sputtering_time):
        
        self.mol_data = dict(pd.read_excel("molecular_database.xlsx")) # dict with molecular volume of elements
        
        self.Element1 = str(Element1)
        self.Element2 = str(Element2)
        self.concentration_Element1 = float(concentration_Element1)
        self.total_thickness = float(total_thickness)
        self.sputtering_time = int(sputtering_time)
        
    def load_sputtering_parameters(self,AK_Value):
        self.sputtering_parameters = pd.read_excel(f"sputtering_parameters_{AK_Value}.xlsx",index_col=[0])
            
    def power_calculation(self,AK_Value):
        self.load_sputtering_parameters(AK_Value)
        tElement1, tElement2 = symbols('tElement1, tElement2')
        eq1 = Eq(100*(tElement1/self.mol_data[self.Element1])/((tElement1/self.mol_data[self.Element1]) + ((tElement2)/self.mol_data[self.Element2])),self.concentration_Element1)
        eq2 =  Eq(tElement2 + tElement1,self.total_thickness)
        solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement1,tElement2)).items())
        self.thickness_Element1  = solved['tElement1'] 
        self.thickness_Element2 = solved['tElement2']
        self.sputtering_rate_Element1 = float(self.thickness_Element1/self.sputtering_time)
        self.sputtering_rate_Element2 = float(self.thickness_Element2/self.sputtering_time)
        try:
            self.power_Element1 = (self.sputtering_rate_Element1-self.sputtering_parameters.loc[self.sputtering_parameters["Element"]==self.Element1,"Intercept"].item())/self.sputtering_parameters.loc[self.sputtering_parameters["Element"]==self.Element1,"Slope"].item()
            self.power_Element2 = (self.sputtering_rate_Element2-self.sputtering_parameters.loc[self.sputtering_parameters["Element"]==self.Element2,"Intercept"].item())/self.sputtering_parameters.loc[self.sputtering_parameters["Element"]==self.Element2,"Slope"].item()
        except ValueError:
            print("Please provide sputtering parameters for specific elements")


In [7]:

class App_window():
    
    def __init__(self, sample):

        self.sample = sample
        
        #root initialization
        self.root = tk.Tk()
        self.root.title("Sputtering rates by Łukasz Frąckowiak")
        self.root.geometry("700x300")
        self.root.resizable(False, False)
        #cordinations initialization
        self.x, self.y = 0.02, 0.05
        self.x2=0.2
        self.test = sample("Co","Au",1,1,1)
        self.start_frame()
        self.create_field()
        self.root.mainloop()
        
        
     
    def start_frame(self):
        self.text = tk.Label(text="Below, please provide information about alloy sample", font=("", 10))
        self.text.place(relx=self.x, rely=self.y)
        
        self.button_exit = tk.Button(text="EXIT", command = self._exit)
        self.button_exit.place(relx=0.9, rely=self.y)
        
        self.button_calculate = tk.Button(text="Calculate", command = self.calculate)
        self.button_calculate.place(relx=0.6, rely=self.y)

    def create_field(self):
        
        self.text_elements = tk.Label(text="Name of elements", font=("", 10))
        self.text_elements.place(relx=self.x, rely=self.y+0.15)
        self.combobox_element1 = Combobox(values=list(self.test.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element1.place(relx= self.x2, rely= self.y+0.15)
        
        self.combobox_element2 = Combobox(values=list(self.test.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element2.place(relx= self.x2+0.1, rely= self.y+0.15)
        
        self.text_AK = tk.Label(text="A-K height [cm]", font=("", 10))
        self.text_AK.place(relx=self.x, rely=self.y+0.3)
        
        self.var = IntVar()
        self.radiobutton_10 = tk.Radiobutton(text="10",variable=self.var,value=10)
        self.radiobutton_10.place(relx=self.x2, rely=self.y+0.3)
        self.radiobutton_15 = tk.Radiobutton(text="15", variable=self.var, value=15)
        self.radiobutton_15.place(relx=self.x2+0.1, rely=self.y+0.3)

        self.text_thickness = tk.Label(text="Alloy thickness [A]", font=("", 10))
        self.text_thickness.place(relx=self.x, rely=self.y+0.45)        
        self.entry_thickness = tk.Entry()
        self.entry_thickness.place(relx= self.x2, rely= self.y+0.45)
        
        self.text_concentration = tk.Label(text="Concentration [at.%]", font=("", 10))
        self.text_concentration.place(relx=self.x, rely=self.y+0.6)        
        self.entry_concentration = tk.Entry()
        self.entry_concentration.place(relx= self.x2, rely= self.y+0.6)
        
        self.text_time = tk.Label(text="Sputtering time [s]", font=("", 10))
        self.text_time.place(relx=self.x, rely=self.y+0.75)        
        self.entry_time = tk.Entry()
        self.entry_time.place(relx= self.x2, rely= self.y+0.75)
        
        
    def calculate(self):
        try:
            Sample = sample(str(self.combobox_element1.get()),str(self.combobox_element2.get()),float(self.entry_concentration.get()),float(self.entry_thickness.get()),int(self.entry_time.get()))
        except ValueError:
            messagebox.showerror("Error 1", "You put wrong parameters to calculation")


        try:
            Sample.power_calculation(self.var.get())
        except AttributeError:
             messagebox.showerror("Error 2", "Please provide molecular_database.xlsx file")            
        except FileNotFoundError:
             messagebox.showerror("Error 3", f"Please check if sputtering_parameters_{self.var.get()}.xlsx file exists")

        try:
            self.text_power_Element1 = tk.Label(text=f"Power of {Sample.Element1} = {Sample.power_Element1} [W]", font=("", 10))
            self.text_power_Element1.place(relx=0.6, rely=0.6)
            self.text_power_Element2 = tk.Label(text=f"Power of {Sample.Element2} = {Sample.power_Element2} [W]", font=("", 10))
            self.text_power_Element2.place(relx=0.6, rely=0.7)
        except AttributeError:
            messagebox.showerror("Error 4", "Please provide sputtering parameters for chosen elements")            
        
    def _exit(self):
        self.root.quit()  # stops mainloop
        self.root.destroy()  # this is necessary on Windows to preven


App_window(sample)

Please provide sputtering parameters for specific elements
